In [1]:
import requests
import base64

# Replace with your own Client ID and Client Secret
CLIENT_ID = 'your_client_id'
CLIENT_SECRET = 'your_client_secret'

# Base64 encode the client ID and client secret
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

# Request the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()

Error obtaining access token.


In [1]:
pip install spotipy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Step 1: Define the function to get playlist data
def get_trending_playlist_data(playlist_id, sp):
    # Get the tracks from the playlist
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')

    # Extract relevant information and store in a list of dictionaries
    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']

        # Get audio features for the track
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

        # Get release date of the album
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None

        # Get popularity of the track
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None

        # Add additional track information to the track data
        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
        }

        music_data.append(track_data)

    # Create a pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(music_data)
    return df

In [3]:
# Step 2: Authenticate with Spotify using your credentials
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id='bb7515607a0e443195d8a421bdf0858c',
                                               client_secret='ba224126f6014b539ecbf1bc9142571e',
                                               redirect_uri='http://localhost:9090',
                                               scope='playlist-read-private'))

In [4]:
# Step 3: Provide the playlist ID and call the function
playlist_id = '2cT3gVyXeD4b8inJOzqUr3'
music_df = get_trending_playlist_data(playlist_id, sp)

In [5]:
print(music_df)

                                        Track Name  \
0                        STAY (with Justin Bieber)   
1                                            Ghost   
2                                             Baby   
3                                          bad guy   
4                I Don't Care (with Justin Bieber)   
5                                What Do You Mean?   
6           Peaches (feat. Daniel Caesar & Giveon)   
7                                            Sorry   
8                                Beauty And A Beat   
9                                    Love Yourself   
10                                           Ghost   
11                       STAY (with Justin Bieber)   
12                               THATS WHAT I WANT   
13  Beautiful Mistakes (feat. Megan Thee Stallion)   
14                                     My Universe   
15                          Mood (feat. iann dior)   
16                                     WITHOUT YOU   

                           

In [6]:
print(music_df.isnull().sum())

Track Name       0
Artists          0
Album Name       0
Album ID         0
Track ID         0
Popularity       0
Release Date     0
Duration (ms)    0
Danceability     0
Energy           0
Loudness         0
Tempo            0
dtype: int64


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

data = music_df

In [8]:
# Function to calculate weighted popularity scores based on release date
def calculate_weighted_popularity(release_date):
    # Convert the release date to datetime object
    release_date = datetime.strptime(release_date, '%Y-%m-%d')

    # Calculate the time span between release date and today's date
    time_span = datetime.now() - release_date

    # Calculate the weighted popularity score based on time span (e.g., more recent releases have higher weight)
    weight = 1 / (time_span.days + 1)
    return weight       

In [10]:
print(music_df.columns)

Index(['Track Name', 'Artists', 'Album Name', 'Album ID', 'Track ID',
       'Popularity', 'Release Date', 'Duration (ms)', 'Danceability', 'Energy',
       'Loudness', 'Tempo'],
      dtype='object')


In [11]:
from sklearn.preprocessing import MinMaxScaler

# Normalize the music features using Min-Max scaling
scaler = MinMaxScaler()
music_features = music_df[['Danceability', 'Energy', 'Loudness', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

# Optionnel : Ajoutez les données normalisées à un nouveau DataFrame si besoin
scaled_features_df = pd.DataFrame(music_features_scaled, columns=['Danceability', 'Energy', 'Loudness', 'Tempo'])
print(scaled_features_df)

    Danceability    Energy  Loudness     Tempo
0       0.066176  0.802495  0.761058  1.000000
1       0.102941  0.754678  0.749868  0.847757
2       0.569853  1.000000  0.793576  0.000000
3       0.356618  0.155925  0.000000  0.667436
4       0.827206  0.619543  0.819379  0.351938
5       1.000000  0.392931  0.414297  0.571836
6       0.382353  0.661123  0.669300  0.238232
7       0.301471  0.794179  1.000000  0.332784
8       0.102941  0.966736  0.841890  0.600267
9       0.132353  0.000000  0.189179  0.337389
10      0.102941  0.754678  0.749868  0.847757
11      0.066176  0.802495  0.761058  1.000000
12      0.602941  0.972973  0.889284  0.218697
13      0.514706  0.619543  0.761190  0.324212
14      0.000000  0.692308  0.657846  0.381017
15      0.470588  0.702703  0.999737  0.247547
16      0.327206  0.072765  0.514481  0.266597


In [12]:
# a function to get content-based recommendations based on music features
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get the index of the input song in the music DataFrame
    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]

    # Calculate the similarity scores based on music features (cosine similarity)
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)

    # Get the indices of the most similar songs
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

    # Get the names of the most similar songs based on content-based filtering
    content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]

    return content_based_recommendations

In [13]:
import pandas as pd

def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)

    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]

    weighted_popularity_score = popularity_score * calculate_weighted_popularity(
        music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0]
    )

    new_entry = pd.DataFrame({
        'Track Name': [input_song_name],
        'Artists': [music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0]],
        'Album Name': [music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0]],
        'Release Date': [music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0]],
        'Popularity': [weighted_popularity_score]
    })

    hybrid_recommendations = pd.concat([content_based_rec, new_entry], ignore_index=True)

    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)

    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]

    return hybrid_recommendations

In [14]:
input_song_name = "Ghost"
recommendations = hybrid_recommendations(input_song_name, num_recommendations=5)
print(f"Hybrid recommended songs for '{input_song_name}':")
print(recommendations)

Hybrid recommended songs for 'Ghost':
                  Track Name                       Artists  \
1  STAY (with Justin Bieber)  The Kid LAROI, Justin Bieber   
2  STAY (with Justin Bieber)  The Kid LAROI, Justin Bieber   
3          Beauty And A Beat    Justin Bieber, Nicki Minaj   
4                My Universe                 Coldplay, BTS   

               Album Name Release Date  Popularity  
1  F*CK LOVE 3+: OVER YOU   2021-07-27        84.0  
2  F*CK LOVE 3+: OVER YOU   2021-07-27        84.0  
3                 Believe   2012-01-01        80.0  
4    Music Of The Spheres   2021-10-15        77.0  
